In [1]:
import trainers.dpo as dpo
from datasets import load_dataset
from torch.optim import AdamW
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)
import torch

model_id = "meta-llama/Llama-3.2-1B"


model = AutoModelForCausalLM.from_pretrained(
            model_id,device_map="cuda")

ref_model = AutoModelForCausalLM.from_pretrained(
            model_id,device_map="cuda")

tokenizer = AutoTokenizer.from_pretrained(model_id)


In [2]:
from trainers import dpo
ds = load_dataset("HumanLLMs/Human-Like-DPO-Dataset")


In [4]:

# 訓練データのサブセットを選択
dataset = ds['train'].select(range(200))
beta = 0.2

dpo_model = dpo.DPO(model_id, model,ref_model,beta)
optimizer = AdamW(dpo_model.model.parameters(), lr=1e-5)

In [5]:

# トレーニング実行
dpo_model.train(dataset, optimizer, num_epochs=5)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

 20%|██        | 1/5 [01:55<07:40, 115.03s/it]

Epoch 1, Loss: nan


 40%|████      | 2/5 [03:49<05:44, 114.85s/it]

Epoch 2, Loss: nan


 60%|██████    | 3/5 [05:44<03:49, 114.93s/it]

Epoch 3, Loss: nan


 60%|██████    | 3/5 [06:53<04:35, 137.73s/it]


KeyboardInterrupt: 

In [ ]:
ds['train'][0]

{'prompt': 'Oh, I just saw the best meme - have you seen it?',
 'chosen': "😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣",
 'rejected': "I'm an artificial intelligence language model, I don't have personal experiences or opinions. However, I can provide you with information on highly-rated and critically acclaimed films, as well as recommendations based on specific genres or themes. Would you like me to suggest some notable movies or discuss a particular genre of interest?"}

In [ ]:
from transformers import AutoTokenizer,set_seed


tokenizer = AutoTokenizer.from_pretrained(model_id)

input_ids = tokenizer.encode(ds['train'][0]["prompt"], return_tensors="pt").to(dpo_model.model.device)

set_seed(54)
output =dpo_model.model.generate(input_ids,do_sample=True,top_p=0.8,max_new_tokens=128)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
print(tokenizer.decode(output[0], skip_special_tokens=True))


Oh, I just saw the best meme - have you seen it? - I, you. I, I just saw it I I just see the I I I just seen it, I I I I I it I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

input_ids = tokenizer.encode(ds['train'][0]["prompt"], return_tensors="pt").to(dpo_model.ref_model.device)

set_seed(54)

output =dpo_model.ref_model.generate(input_ids,do_sample=True,top_p=0.8,max_new_tokens=128)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(tokenizer.decode(output[0], skip_special_tokens=True))


Oh, I just saw the best meme - have you seen it? - and the most annoying thing that ever happened to my life. A bunch of people are here to get me started on the next generation of things. We all got this shit together and all of a sudden we all get to be this big. I had a lot of fun, I really did, and I had my first crush on someone who was in their 30s. I remember my dad, my sister, and my mom were out in the car on the way to work and we were like, "Whoa! I got my first crush on a guy who is 50!" It was the coolest thing that ever happened to me,
